In [1]:
import numpy as np
import random
from IPython.display import clear_output
from collections import deque
import progressbar

import gym
import gym_gvgai
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape, Input, Flatten
from tensorflow.keras.optimizers import Adam

from datetime import datetime
from packaging import version

# Constants

IMG_H = 9
IMG_W = 13
STATE_SIZE = (IMG_H, IMG_W)

# Tensorboard extension

%load_ext tensorboard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

class Agent:
    def __init__(self, enviroment):
        
        # Initialize atributes
        self._state_size = STATE_SIZE
        #self._state_size = enviroment.observation_space.n
        self._action_size = enviroment.action_space.n
        
        self.expirience_replay = deque(maxlen=2000)
        
        # Initialize discount and exploration rate
        self.gamma = 0.6
        self.epsilon = 0.1
        
        # Build networks
        self.q_network = self._build_compile_model()
        self.target_network = self._build_compile_model()
        self.align_target_model()

    def store(self, state, action, reward, next_state, terminated):
        self.expirience_replay.append((state, action, reward, next_state, terminated))
    
    def _build_compile_model(self):
        model = Sequential()
        model.add(Flatten(input_shape=STATE_SIZE))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=0.1))
        return model

    def align_target_model(self):
        self.target_network.set_weights(self.q_network.get_weights())
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return enviroment.action_space.sample()
        
        tensor_state = tf.convert_to_tensor([state])
        
        q_values = self.q_network.predict(tensor_state)
        return np.argmax(q_values[0])

    def retrain(self, batch_size):
        minibatch = random.sample(self.expirience_replay, batch_size)
        
        for state, action, reward, next_state, terminated in minibatch:
            
            tensor_state = tf.convert_to_tensor([state])
            tensor_next_state = tf.convert_to_tensor([next_state])
            
            q_values = self.q_network.predict(tensor_state)
            
            if terminated:
                q_values[0][action] = reward
            else:
                t = self.target_network.predict(tensor_next_state)
                q_values[0][action] = reward + self.gamma * np.amax(t)
            
            self.q_network.fit(tensor_state, q_values, epochs=1, verbose=0, callbacks=[tensorboard_callback])
            
    def save_model(self, model_name="q_network"):
        self.q_network.save(model_name)
        
    def load_model(self, model_path):
        self.q_network = keras.models.load_model(model_path)
        self.align_target_model()

TensorFlow version:  2.4.1


In [2]:
# Utils

def grayToArray(array):
    result = np.zeros((IMG_H, IMG_W))
    for i in range(int(array.shape[0]/10)):
        for j in range(int(array.shape[1]/10)):
            result[i][j] = int(array[10*i+5, 10*j+5])
    return result


def grayConversion(image):
    b = image[..., 0]
    g = image[..., 1]
    r = image[..., 2]
    return 0.21 * r + 0.72 * g + 0.07 * b

def reshape_state(state):
    return grayToArray(grayConversion(state))

In [3]:
# Train

batch_size = 32
num_of_episodes = 100
timesteps_per_episode = 1000

enviroment = gym.make("gvgai-zelda-lvl0-v0")
agent = Agent(enviroment)
agent.q_network.summary()

Connecting to host 127.0.0.1 at port 55519 ...
Client connected to server [OK]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 117)               0         
_________________________________________________________________
dense (Dense)                (None, 24)                2832      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 150       
Total params: 3,582
Trainable params: 3,582
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for e in range(0, num_of_episodes):
    # Reset the enviroment
    state = enviroment.reset()
    state = reshape_state(state)
    
    # Initialize variables
    reward = 0
    terminated = False
    
    bar = progressbar.ProgressBar(maxval=timesteps_per_episode/10, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    for timestep in range(timesteps_per_episode):
        # Run Action
        action = agent.act(state)
        
        # Take action    
        next_state, reward, terminated, info = enviroment.step(action) 
        next_state = reshape_state(next_state)
        agent.store(state, action, reward, next_state, terminated)
        
        state = next_state
        
        if terminated:
            agent.align_target_model()
            break
            
        if len(agent.expirience_replay) > batch_size:
            agent.retrain(batch_size)
        
        if timestep%10 == 0:
            bar.update(timestep/10 + 1)
    
    bar.finish()
    if (e + 1) % 10 == 0:
        print("**********************************")
        print("Episode: {}".format(e + 1))
        enviroment.render()
        print("**********************************")
        agent.save_model("zelda_2_q_network_{}".format(e))

[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%


In [36]:
# Play

enviroment = gym.make("gvgai-zelda-lvl0-v0")
agent = Agent(enviroment)

agent.load_model("zelda_q_network_1")

state = enviroment.reset()
state = reshape_state(state)

for timestep in range(2000):
    action = agent.act(state)
    next_state, reward, terminated, info = enviroment.step(action)
    print(action, reward, terminated, info)
    state = reshape_state(next_state)
    if terminated:
        print("terminated")
        break

Connecting to host 127.0.0.1 at port 52304 ...
Client connected to server [OK]


OSError: SavedModel file does not exist at: zelda_q_network_1/{saved_model.pbtxt|saved_model.pb}